In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_theme()

import torch 
import torch.nn as nn 
import torch.optim as optim


from models.learned_qp_solver_bi_cycle_vel_part import MLP_Init, learned_qp_solver
from scipy.io import loadmat
from torch.utils.data import Dataset, DataLoader
# torch.set_default_dtype(torch.float64)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [3]:


num = 50
t = 0.1


vel_min = 0.0
vel_max = 30.0

acc_min = -6.0
acc_max = 6.0

jerk_min = -6.0
jerk_max = 6.0



In [4]:
#### creating dataset 

dataset_size = 200000

vel_samples = np.random.uniform(vel_min, vel_max,  (dataset_size, num)    )
# vel_samples = np.random.multivariate_normal(np.zeros(num), vel_max*np.identity(num), (dataset_size, ) )

vel_samples = np.clip( vel_samples, vel_min, vel_max  )



vel_init = np.random.uniform(vel_min, vel_max, (dataset_size, 1)  )


inp = np.hstack(( vel_samples,  vel_init ))

inp_min = inp.min()
inp_max = inp.max()

# normalization_const = (vel_max-vel_min)

inp_mean, inp_std = inp.mean(), inp.std()

median_ = np.median(inp, axis=0)
q1 = np.percentile(inp, 25, axis=0)
q3 = np.percentile(inp, 75, axis=0)
iqr_ = q3 - q1


# Handle constant features
iqr_ = np.where(iqr_ == 0, 1, iqr_)

print(np.array_repr(median_))
print(np.array_repr(iqr_))


# inp_mean = inp_min 
# inp_std = inp_max


median_ = torch.tensor(median_).float().to(device)
iqr_ = torch.tensor(iqr_).float().to(device)



print(inp_mean, inp_std)


array([15.04770936, 15.05825353, 15.05594789, 14.98925399, 14.9813385 ,
       14.9803559 , 15.01201724, 15.05104337, 14.99071299, 14.99062193,
       14.97814039, 14.98633039, 15.05541669, 15.00973865, 14.97581504,
       14.95434159, 15.08204415, 14.95716258, 15.00798985, 15.02159146,
       15.00003375, 14.99012723, 15.03607882, 15.04781101, 15.02108329,
       14.99038303, 14.95707157, 15.04866477, 14.95856117, 15.01981668,
       15.03673595, 15.06312754, 15.01430138, 14.99278037, 14.95117927,
       15.00640649, 15.05287354, 15.01712211, 15.03036029, 14.96818817,
       14.9910865 , 14.97771491, 14.96045441, 14.96508377, 15.01555771,
       15.00657147, 15.01346687, 14.940316  , 14.94994511, 14.93062279,
       14.98692683])
array([14.9777    , 15.00083603, 14.97139699, 15.04253366, 14.93629004,
       14.98271924, 14.97111582, 14.99223769, 14.99461593, 15.04091625,
       14.98874495, 14.9530227 , 15.00100103, 15.02562935, 15.03006681,
       14.93371082, 14.98533277, 14.9988090

In [5]:
# Custom Dataset Loader 
class TrajDataset(Dataset):
	"""Expert Trajectory Dataset."""
	def __init__(self, inp, vel_init, vel_samples):
		
		# State Data
		self.inp = inp
  
		self.vel_init = vel_init
		
		# Expert Coeff
		self.vel_samples = vel_samples

		
		
	
	def __len__(self):
		return len(self.inp)    
			
	def __getitem__(self, idx):
		
		# Inputs
		inp = self.inp[idx]
		vel_init = self.vel_init[idx]

		
		# Outputs
		vel_samples = self.vel_samples[idx]
	
		
		
		return torch.tensor(inp).float(), torch.tensor(vel_init).float(), torch.tensor(vel_samples).float()
	

# Batch Size - 3k or 4k
batch_size = 2056

# pcd_data = pcd_data.reshape(data_set_size, 2, 200)

# Using PyTorch Dataloader
train_dataset = TrajDataset(inp, vel_init,  vel_samples)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)



In [6]:
# Differentiable Layer
num_batch = train_loader.batch_size


mlp_init_inp_dim = np.shape(inp)[1]
mlp_init_hidden_dim = 1024
# mlp_init_out_dim = (2*nvar)**2+2*nvar+2*nvar
mlp_init_out_dim = num+(2*num+2*(num-1)+2*(num-2))

# mlp_pred = MLP_Pred(mlp_pred_inp_dim, mlp_pred_hidden_dim, mlp_pred_out_dim  )
mlp_init = MLP_Init(mlp_init_inp_dim, mlp_init_hidden_dim, mlp_init_out_dim  )


model = learned_qp_solver(num_batch, num, t, mlp_init).to(device)
# model.load_state_dict(torch.load('./weights/learned_qp_bi_cycle_vel_part.pth'))



In [7]:
epochs = 800
step = 0 
beta = 1.0 # 3.5

optimizer = optim.AdamW(model.parameters(), lr = 1e-4, weight_decay=6e-5)
# optimizer = optim.RMSprop(model.parameters(), lr = 1e-3, weight_decay=6e-5)

# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 100, gamma = 0.1)

avg_train_loss, avg_loss_primal, avg_fixed_point_loss, avg_proj_loss = [], [], [], []
model.train()
for epoch in range(epochs):
	
	# Train Loop
	losses_train, aug_losses_primal, aug_losses_fixed_point, aug_losses_proj = [], [], [], []
	
	for (inp, vel_init, vel_samples) in train_loader:
		
		# Input and Output 
  
		################################################################################################

		# print(goal_des[0])
		# print(y_lb[0], y_ub[0])

  
		inp = inp.to(device)
		vel_init = vel_init.to(device)
		
		vel_samples = vel_samples.to(device)
	
		
		vel_projected, res_primal_stack, res_fixed_point_stack, accumulated_res_primal, accumulated_res_fixed_point = model(inp, vel_init, vel_samples, vel_max, vel_min, acc_max, acc_min, jerk_max, jerk_min, inp_min, inp_max, median_, iqr_)
		loss, primal_loss, fixed_point_loss, proj_loss = model.ss_loss(accumulated_res_primal, accumulated_res_fixed_point, vel_projected, vel_samples)
	
		optimizer.zero_grad()
		loss.backward()
		# torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
		optimizer.step()
		
		losses_train.append(loss.detach().cpu().numpy()) 
		aug_losses_primal.append(primal_loss.detach().cpu().numpy())
		aug_losses_fixed_point.append(fixed_point_loss.detach().cpu().numpy())
		aug_losses_proj.append(proj_loss.detach().cpu().numpy())
		# aug_losses_steer.append(steer_loss.detach().cpu().numpy())

		
	# scale = scale*1.2	
		
	if epoch % 4 == 0:    
		print(f"Epoch: {epoch + 1}, Train Loss: {np.average(losses_train):.3f}, aug_loss_primal: {np.average(aug_losses_primal):.3f}, aug_loss_fixed_point: {np.average(aug_losses_fixed_point):.3f}, aug_loss_proj: {np.average(aug_losses_proj):.3f} ")

	step += 0.15 #0.15
	# scheduler.step()
	avg_train_loss.append(np.average(losses_train)), avg_loss_primal.append(np.average(aug_losses_primal)), avg_fixed_point_loss.append(np.average(aug_losses_fixed_point)),  avg_proj_loss.append(np.average(aug_losses_proj))
	
		



Epoch: 1, Train Loss: 60.025, aug_loss_primal: 0.674, aug_loss_fixed_point: 18.845, aug_loss_proj: 40.506 
Epoch: 5, Train Loss: 44.700, aug_loss_primal: 0.149, aug_loss_fixed_point: 2.908, aug_loss_proj: 41.643 
Epoch: 9, Train Loss: 43.270, aug_loss_primal: 0.119, aug_loss_fixed_point: 1.654, aug_loss_proj: 41.497 
Epoch: 13, Train Loss: 42.780, aug_loss_primal: 0.089, aug_loss_fixed_point: 1.302, aug_loss_proj: 41.389 
Epoch: 17, Train Loss: 42.166, aug_loss_primal: 0.046, aug_loss_fixed_point: 0.990, aug_loss_proj: 41.130 
Epoch: 21, Train Loss: 41.808, aug_loss_primal: 0.026, aug_loss_fixed_point: 0.767, aug_loss_proj: 41.015 
Epoch: 25, Train Loss: 41.602, aug_loss_primal: 0.019, aug_loss_fixed_point: 0.628, aug_loss_proj: 40.954 
Epoch: 29, Train Loss: 41.482, aug_loss_primal: 0.016, aug_loss_fixed_point: 0.560, aug_loss_proj: 40.906 
Epoch: 33, Train Loss: 41.391, aug_loss_primal: 0.014, aug_loss_fixed_point: 0.514, aug_loss_proj: 40.863 
Epoch: 37, Train Loss: 41.324, aug_loss

In [8]:
torch.save(model.state_dict(), './weights/learned_qp_bi_cycle_vel_part.pth')